### https://webdevblog.ru/gensim-rukovodstvo-dlya-nachinajushhih/
### модель https://rusvectores.org/ru/models/

# FastText TransferLerning


## Загрузка модели

In [16]:
import compress_fasttext

fast_text = compress_fasttext.models.CompressedFastTextKeyedVectors.load(
        './fast_text_model/model.model'
    )

## Оценка предобученной модели

In [17]:
import pandas as pd

train = pd.read_parquet('data_fusion_train.parquet', engine='pyarrow')
train = train[train.category_id != -1].drop_duplicates('item_name')

In [18]:
import re

product_names_arr = [[s for s in re.sub("[^а-я$]", " ", i.lower()).split(' ') if len(s) > 1] for i in train["item_name"].tolist()]


In [19]:
from tqdm import tqdm

product_embenddings_arr = []
for product_names in tqdm(product_names_arr):
    product_embenddings_arr.append([fast_text[i] for i in product_names])

100%|██████████| 48225/48225 [00:04<00:00, 10683.86it/s]


In [20]:
sum_of_embds = [sum(i) for i in product_embenddings_arr]

In [21]:
from sklearn.neighbors import KNeighborsClassifier

data = [i.tolist() if not isinstance(i, int) else [0]*300 for i in sum_of_embds]
labels = train["category_id"].tolist()


In [22]:
classifier = KNeighborsClassifier()
classifier.fit(data, labels)

KNeighborsClassifier()

In [24]:
from sklearn.metrics import classification_report

report = classification_report(classifier.predict(data), labels)

KeyboardInterrupt: 

In [ ]:
print(report)

## Визуализация

In [ ]:
from sklearn import manifold

tsne = manifold.TSNE(n_components = 2, init = 'pca', random_state = 0)
data_2d_tsne = tsne.fit_transform(data)

In [ ]:
from matplotlib import pyplot as PLT
%pylab inline

In [ ]:
pylab.figure(figsize = (10, 6))
pylab.scatter(data_2d_tsne[:, 0], data_2d_tsne[:, 1], c = labels)

# Подготовка данных для обучения 

In [29]:
import pandas as pd

train_data_fast_text = pd.read_parquet('data_fusion_train.parquet', engine='pyarrow')


In [30]:
train_data_fast_text = train_data_fast_text[train_data_fast_text.category_id == -1].drop_duplicates('item_name')


In [32]:
product_names_arr = [[s for s in re.sub("[^а-я$]", " ", i.lower()).split(' ') if len(s) > 1] for i in tqdm(train_fast_text["item_name"].tolist())]


100%|██████████| 26094077/26094077 [04:21<00:00, 99699.21it/s] 


In [33]:
train_fast_text[:10]

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
0,7,5,09:50,Флуконазол капс 150мг №1 Вертекс,1.0,7,-1,-1,вертекс
1,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,None
2,39,4,11:28,"Борщ Станичный с тушенкой, 103 ккал (завод, з/у)",1.0,6,1,-1,None
3,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,None
4,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,None
6,39,4,11:28,"Филе Горизонт (филе птицы), 218 ккал",1.0,7,1,-1,горизонт
10,49,3,14:27,3 Бумага Упак 1/Прочи/В асс,1.0,8,1,-1,None
11,49,3,14:27,4 Бумага Упак 1/Прочи/В асс,1.0,8,1,-1,None
12,49,3,14:27,Брюки спорт мал SAMO M-1016,1.0,13,6,-1,None
13,49,3,14:27,Бюстгальтер БН137,1.0,13,-1,-1,None


## Обучение

### Embendding до обучения

In [34]:
fast_text['хлеб']

array([-1.83336094e-01, -1.25943897e-02, -2.97685176e-01, -2.88096607e-01,
        9.81020555e-02,  6.05299249e-02,  4.04848829e-02,  2.12353200e-01,
       -2.95661718e-01, -2.16925338e-01,  3.59558165e-02, -3.22789788e-01,
       -2.27806624e-02, -1.96019411e-01, -2.56669581e-01,  2.62878835e-01,
       -3.80497351e-02, -3.28662217e-01, -2.01166674e-01, -2.06913158e-01,
        2.22783804e-01,  1.93487369e-02, -2.76149511e-01,  1.71788186e-01,
       -9.44577008e-02,  8.49458426e-02, -1.67114750e-01,  3.18848401e-01,
       -1.05811790e-01, -2.64632404e-01,  7.42099062e-02, -1.91953242e-01,
        4.12224472e-01, -2.93160856e-01,  7.17218965e-03, -1.22161414e-02,
        3.00891511e-02,  2.45449826e-01, -2.60772347e-01,  9.71184075e-02,
       -4.01285999e-02,  3.18790525e-02, -3.60920966e-01,  1.55319929e-01,
       -2.15977803e-02,  1.74923763e-01, -1.52541967e-02,  8.29191655e-02,
       -1.15341023e-01, -3.95677537e-02, -1.42998070e-01,  2.68362939e-01,
       -1.45388069e-02,  

AttributeError: 'CompressedFastTextKeyedVectors' object has no attribute 'train'

### Embendding после обучения

In [ ]:
fast_text['хлеб']